In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,timedelta
mpl.rcParams['figure.figsize'] = (18,12)

import psycopg2
import sys

import math

import plotly.express as px
import plotly.graph_objects as go

import holidays

sns.set()
sns.set_style('whitegrid')

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import STL
from sklearn import linear_model as lm

import pickle

In [116]:
class Node():
    
    def __init__(self,feature_idx,pivot_value,linear_model,depth,leaf_count):
        
        self.feature_idx = feature_idx
        self.pivot_value = pivot_value
        self.model = linear_model
        self.depth = depth
        self.leaf_count = leaf_count
        self.left_node = None
        self.right_node = None  

In [189]:
class LinearModelTree():
    
    def __init__(self,reg_features,max_depth=10,min_samples_split=100,min_samples_leaf=50,
                 model_type='Ridge',num_cat=2,num_cont=100):
        
        self.reg_features = reg_features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.model_type = model_type
        self.num_cat = num_cat
        self.num_cont = num_cont
             
    
    def build_tree(self,X,y,depth=0,leaf_count=0):
        
        if X.shape[0] >= self.min_samples_split and depth <= self.max_depth:
            split_dict = self.best_split(X,y)
            node = Node(split_dict['feature'],split_dict['pivot_value'],split_dict['model'],depth,leaf_count)

            if split_dict['feature'] != 'Leaf':
                (X_l,y_l),(X_r,y_r) = split_dict['split_data']
                node.left_node = self.build_tree(X_l,y_l,depth+1,leaf_count)
                node.right_node = self.build_tree(X_r,y_r,depth+1,leaf_count)
                node.depth = depth+1
        
        else:
            reg_model = lm.Ridge().fit(X[:,self.reg_features].reshape(-1,len(self.reg_features)),y.reshape(-1,1))
            node = Node('Leaf','None',reg_model,depth,leaf_count+1)
            
        return node
    
    
    def best_split(self,X,y):
    
        features_total = X.shape[1]
        r2 = []
        feat_pivot = []
        model_l_store = []
        model_r_store = []
        
        for feature_idx in range(0,features_total):
            
            cat = self.feature_type(X,self.num_cat)
            
            if feature_idx in cat:
                pivot_values = np.unique(X[:,feature_idx])
            else:
                pivot_values = np.linspace(X[:,feature_idx].min(),X[:,feature_idx].max(),self.num_cont)
            
            for pivot_value in pivot_values:
                
                (X_l,y_l),(X_r,y_r) = self.split_node(X,y,feature_idx,pivot_value)
                
                if X_l.shape[0] >= self.min_samples_leaf and X_r.shape[0] >= self.min_samples_leaf:
                    
                    linear_model = lm.Ridge()
                    
                    model_l = linear_model.fit(X_l[:,self.reg_features].reshape(-1,len(self.reg_features)),y_l.reshape(-1,1))
                    
                    linear_model = lm.Ridge()
                    
                    model_r = linear_model.fit(X_r[:,self.reg_features].reshape(-1,len(self.reg_features)),y_r.reshape(-1,1))
                    
                    model_l_store.append(model_l)
                    model_r_store.append(model_r)

                    avg_r2 = 0.5*np.abs(model_l.score(X_l[:,self.reg_features],y_l)) + 0.5*np.abs(model_r.score(X_r[:,self.reg_features],y_r))
                    r2.append(avg_r2)
                        
                    feat_pivot.append((feature_idx,pivot_value))
        
        try:
            feature_idx,pivot_value = feat_pivot[np.argmax(r2)]
            return {'feature':feature_idx,'pivot_value':pivot_value,'split_data':self.split_node(X,y,feature_idx,pivot_value),'model':(model_l_store[np.argmax(r2)],model_r_store[np.argmax(r2)])}
        except:
            return {'feature':'Leaf','pivot_value':'None','split_data':'NoSplit','model':lm.Ridge().fit(X[:,self.reg_features].reshape(-1,len(self.reg_features)),y.reshape(-1,1))}

            
    def split_node(self,X,y,feature_idx,pivot_value):
        
        cat = self.feature_type(X,self.num_cat)
    
        if feature_idx in cat:

            left_node = (X[X[:,feature_idx]!=pivot_value],y[X[:,feature_idx]!=pivot_value])
            right_node = (X[X[:,feature_idx]==pivot_value],y[X[:,feature_idx]==pivot_value])

        else:
            
            index = X[:,feature_idx] >= pivot_value
            left_node = (X[~index],y[~index])
            right_node = (X[index],y[index])

        return left_node,right_node
    
    @staticmethod
    def feature_type(X,num_cat):
        
        cat = []
        features_total = X.shape[1]
        
        for feature_idx in range(0,features_total):
            if np.unique(X[:,feature_idx]).shape[0] <= num_cat:
                cat.append(feature_idx)
                
        return cat  
        
        
    def fit(self,X,y):
        
        self.final_tree = self.build_tree(X,y)
        
        return self
        
        
    def predict_one(self,X,cat):
            
        mytree = self.final_tree

        while mytree.left_node:
            if mytree.feature_idx in cat:
                if X[mytree.feature_idx] == mytree.pivot_value:
                    mytree = mytree.right_node
                else:
                    mytree = mytree.left_node
            else:
                if X[mytree.feature_idx] >= mytree.pivot_value:
                    mytree = mytree.right_node
                else:
                    mytree = mytree.left_node
                    
        return mytree.model.predict(X[self.reg_features].reshape(-1,len(self.reg_features)))
        
    
    def predict(self,X):    
        
        cat = self.feature_type(X,self.num_cat)
        predictions = []
        
        for Xi in X:
            predictions.extend(self.predict_one(Xi,cat))
        
        return predictions
    
    
    def RMSE(self,X,y):
        
        return np.sqrt((1/X.shape[0])*np.sum(np.square(self.predict(X)-y.reshape(-1,1))))
    
    
    def get_depth(self):
        
        return self.final_tree.depth
    
    
    def get_leaves(self):
        
        return self.final_tree.leaf_count
    
    
    def get_params(self):
        
        pass
    
    
    def path_taken(self):
        
        tree_ = self.final_tree
        path_taken_ = [('Root',tree_.feature_idx,tree_.pivot_value)]
        loc = 'Left'
        
        while tree_.left_node:
            tree_ = tree_.left_node
            if loc == 'Left':
                path_taken_.append((loc,tree_.feature_idx,tree_.left_node.pivot_value))
                loc = 'Right'
                tree_ = tree_.right_node
            if loc == 'Right':
                path_taken_.append((loc,tree_.right_node.feature_idx,tree_.right_node.pivot_value))
                loc = 'Left'
                
        return path_taken_
            

In [109]:
param_dic = {
    "host"      : "localhost",
    "database"  : "demand",
    "user"      : "postgres",
    "password"  : "password"
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    tupples = cursor.fetchall()
    cursor.close()
    
    data = pd.DataFrame(tupples, columns=column_names)
    return data

In [110]:
conn = connect(param_dic)

column_names = ['timestamp', 'hour', 'temp_index', 'week_index', 'temperature', 'demand']

query = ("SELECT temp_data.timestamp,hour,temp_index,week_index,temperature,demand_data.demand" 
         " FROM demand_data LEFT JOIN temp_data ON temp_data.timestamp = demand_data.timestamp"
         " ORDER BY timestamp")

data = postgresql_to_dataframe(conn, query, column_names)
data.set_index('timestamp',inplace=True)

Connecting to the PostgreSQL database...
Connection successful


In [111]:
data.dropna(inplace=True)
display(data.head())
display(data.shape)

,hour,temp_index,week_index,temperature,demand
timestamp,,,,,
2018-01-01 00:00:00,0.0,NotHot,Weekday,-19.5,5340.0
2018-01-01 01:00:00,1.0,NotHot,Weekday,-21.7,5211.0
2018-01-01 02:00:00,2.0,NotHot,Weekday,-19.3,5096.0
2018-01-01 03:00:00,3.0,NotHot,Weekday,-20.3,4987.0
2018-01-01 04:00:00,4.0,NotHot,Weekday,-19.9,4926.0


(27501, 5)

In [112]:
def deal_with_dayofweek(x):
    if x == 'Weekday':
        return 0
    else:
        return 1

In [113]:
data.drop('temp_index',axis=1,inplace=True)
data['week_index'] = data['week_index'].apply(deal_with_dayofweek)
df = data.to_numpy()
X = df[:,:-1]
y = df[:,-1]

In [190]:
mylineartree = LinearModelTree(reg_features=[2],num_cat=24)

In [191]:
model = mylineartree.fit(X[:26000],y[:26000])

In [192]:
model.predict(X[26000:])

[array([5820.06512771]),
 array([5806.7427416]),
 array([5793.42035549]),
 array([5846.70989993]),
 array([5846.70989993]),
 array([5846.70989993]),
 array([5828.94671845]),
 array([5837.82830919]),
 array([6263.48557949]),
 array([5765.14833982]),
 array([5535.65935325]),
 array([5262.90135777]),
 array([4949.82399279]),
 array([4738.81099209]),
 array([4531.10350779]),
 array([4401.29641889]),
 array([4345.92788996]),
 array([4296.94818927]),
 array([5820.06512771]),
 array([5820.06512771]),
 array([5806.7427416]),
 array([5797.86115086]),
 array([5788.97956012]),
 array([5775.65717402]),
 array([5771.21637865]),
 array([5753.45319717]),
 array([5709.04524347]),
 array([5686.84126662]),
 array([5700.16365273]),
 array([5713.48603884]),
 array([5722.36762958]),
 array([5722.36762958]),
 array([6103.90365604]),
 array([5606.83414334]),
 array([5419.32528907]),
 array([5171.12572596]),
 array([4896.32864793]),
 array([4909.84231352]),
 array([4790.65501398]),
 array([4603.8815766]),
 ar

In [193]:
model.RMSE(X[26000:],y[26000:])

497.35161274537876

In [199]:
tree_ = model.final_tree
path_taken_ = [('Root',tree_.feature_idx,tree_.pivot_value)]
loc = 'Left'

while tree_.left_node:
    tree_ = tree_.left_node
    if loc == 'Left':
        try:
            path_taken_.append((loc,tree_.feature_idx,tree_.left_node.pivot_value))
        except:
            continue
        loc = 'Right'
        tree_ = tree_.right_node
    if loc == 'Right':
        try:
            path_taken_.append((loc,tree_.feature_idx,tree_.pivot_value))
        except:
            continue
        loc = 'Left'


In [200]:
path_taken_

[('Root', 2, 15.533333333333331),
 ('Left', 0, 1.0),
 ('Right', 1, 0.0),
 ('Left', 2, 'None'),
 ('Right', 2, 8.527272727272727),
 ('Left', 2, 'None'),
 ('Right', 'Leaf', 'None')]